<a href="https://colab.research.google.com/github/GauravBhardwaj007/Coversational-RAG/blob/main/Conversational_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install langchain
# %%
!pip install -U langchain-community

# %%
# Install pypdf BEFORE importing and using PyPDFLoader
!pip install pypdf

# %%




0.3.25
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
import langchain

In [ ]:
print(langchain.__version__)

0.3.25


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
import os


In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/Projects/RAG/Carbon-Offset-Guide_3122020.pdf")
documents = loader.load()


In [ ]:
# Define the list of PDF file paths
# Make sure these paths are correct and the files exist
pdf_files = [
    "/content/drive/MyDrive/Projects/RAG/Carbon-Offset-Guide_3122020.pdf",
    # Add other PDF file paths here, e.g.,
    "/content/drive/MyDrive/Projects/RAG/carbon-offsets.pdf",
    # "/content/drive/MyDrive/Projects/RAG/third_document.pdf",
]

# Load documents from all PDFs
all_documents = []
for pdf_file in pdf_files:
    if os.path.exists(pdf_file):
        loader = PyPDFLoader(pdf_file)
        all_documents.extend(loader.load())
    else:
        print(f"Warning: File not found at {pdf_file}")

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=50)
chunks = text_splitter.split_documents(all_documents)


Load a free Vector embedding ModernTreasuryLoader

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-14-357e580d7132>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

store those embeddings in a vector database using FAISS, which is fast, local, and doesn’t need any API.

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 34.2 MB/s eta 0:00:00


In [ ]:
# FAISS vectorstore

from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(chunks, embeddings)

Load a lightweight local LLM (e.g. TinyLlama)
We'll use Hugging Face's free models.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cpu


Create the RAG chain (retriever + LLM)

In [ ]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_kwargs={"k": 3})

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)


In [ ]:
query2 = "What is carbon offset in two lines?"
result2 = rag_chain(query2)

print("Answer 2:", result2['result'])

Answer 2: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

What are Carbon Offsets?
 
Carbon offsets
​ 
 are a “credit for negating or diminishing the impact of emitting a ton of carbon
 
dioxide by paying someone else to absorb or avoid the release of a ton of CO
​ 
2
​ 
 elsewhere.”
 
Simply being alive entails that your individual effect on the planet is carbon-positive (because
 
animals such as humans emit CO
​ 
2
​ 
, if for no other reason), and this effect increases with the use
 
of electricity from non-renewable sources, transportation involving fossil fuels (particularly air
 
and car travel), and all the supporting industries and infrastructures that themselves are carbon
 
positive (i.e., purchasing red meat, or indeed most products). However, by paying for the
 
reduction of carbon emissions elsewhere, you can reduce the net effect of some or all of these
 
a